In [1]:
%%time  
import gc
from sklearn.metrics import mean_squared_error  
from sklearn.model_selection import train_test_split  
from sklearn.preprocessing import StandardScaler  
from keras.models import Sequential  
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam

from keras.callbacks import EarlyStopping,ReduceLROnPlateau

X = df_cleaned.drop(columns="bg+1:00")  
y = df_cleaned['bg+1:00']  

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardizing the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# ANN model
ann_model = Sequential()
ann_model.add(Dense(64, input_dim=X_train_scaled.shape[1], activation='relu'))
ann_model.add(Dropout(0.3))  # Dropout for regularization
ann_model.add(Dense(32, activation='relu'))
ann_model.add(Dropout(0.3))  # Another dropout layer
ann_model.add(Dense(1))

# Compile the model
ann_model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error')

# Early stopping and learning rate scheduler
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)

# Clear memory
gc.collect()

# Train the model with early stopping and learning rate scheduler
history = ann_model.fit(X_train_scaled, y_train, epochs=100, batch_size=10, 
                        validation_split=0.2, callbacks=[early_stopping, lr_scheduler])

# Predictions and RMSE calculation  
ann_predictions = ann_model.predict(X_test_scaled)  
ann_rmse = np.sqrt(mean_squared_error(y_test, ann_predictions))  
gc.collect()

NameError: name 'df_cleaned' is not defined